In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Professor names
professor_names = ['김유섭', '김은주', '이정근', '양은샘', '신미영', '김선정']

### “모르겠다”, “수업을 안들어서 모른다”, “기억 안남” 등의 응답을 NaN 처리하고 결측값 제거하기

1. **문장 유사도 기반 NaN 처리**  
   - `SentenceTransformer`를 이용해 기준 문장(“모르겠다”, “들어본 적 없음”, “기억 안난다” 등)을 먼저 임베딩하여 저장  
   - 각 리뷰 문장에 대해 동일한 임베딩 모델로 벡터화하고, 기준 임베딩들과의 코사인 유사도를 계산  
   - 유사도가 임계치(예: 0.85) 이상인 경우 `np.nan`으로 반환하여 “모름” 응답을 결측값으로 표시  

2. **결측값 제거**  
   - Pandas의 `dropna()` 또는 `DataFrame.dropna(axis=0, how='any')` 메서드를 사용해 NaN이 된 행(응답)들을 제거  
   - 이렇게 하면 실제 수업을 들은 경험에 기반한 리뷰만 남겨, 이후 분석의 왜곡을 막습니다  

In [2]:
df = pd.read_excel('professor_review.xlsx')

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df.head()

,응답일시,참여자,김유섭 교수님,양은샘 교수님,신미영 교수님,김선정 교수님,이정근 교수님,김은주 교수님
0,2025-05-13 18:51:32,1,"친근하다,적극적이다,좋다","친근하다,적극적이다,좋다","친근하다,적극적이다,좋다","친근하다,적극적이다,좋다","친근하다,적극적이다,좋다","친근하다,적극적이다,좋다"
1,2025-05-13 18:56:29,2,NaN,NaN,NaN,매 수업마다 pdf파일을 올려주시며 그 pdf에 나온 대로 따라만 하면 어느정도 수...,NaN,NaN
2,2025-05-13 19:33:43,3,교수님의 말투가 친근하고 또박또박 말씀하셔서 듣기가 좋습니다. 강의 자료만 봐도 이...,과제가 좀 많은게 단점이지만 그만큼 배워 갈수있는것도 많은 교수님이신것같습니다. 수...,강의 자료가 좋아서 교수님중에서 가장 배워갈께 많은 교수님인것같습니다. 다만 신미영...,NaN,이정근 교수님은 굉장히 친절하십니다. 강의자료가 영어로 되어있고 보기 힘들었지만 수...,NaN
3,2025-05-13 20:01:56,4,말투가 느리시지만 상냥한 어투로 말해주신다. 강의 자료는 깔끔하고 이해가 쉽게 되어...,"학생들에게 친근하게 대해주시고, 수업 시에 딕션이 좋으시다. 강의 자료는 이해하기 ...",교수님 말투가 전달력이 좋으신 편은 아니시다. 강의 자료는 이해하기 쉽게 깔끔하게 ...,"수업 속도가 빠른 편이고, 이해하기 쉽게 설명해 주신다. 과제는 매주 있긴 하지만 ...","말이 느리시지만, 학생들에게 친절하게 말해주신다. 수업은 이해가 쉽도록 학생들 입장...",딕션이 좋으시고 말이 굉장히 빠르시다. 말이 빠르셔서 그냥 귀에서 스쳐갈 때도 있지...
4,2025-05-13 20:05:14,5,NaN,수업 내용이 많지만 그만큼 알차다. 과제는 짧으면 1시간 오래걸리면 2~3시간까지 ...,NaN,NaN,NaN,"말투가 굉장이 빠르시고, 강의는 쉬운 편이다. 과제량도 매주 있지만 1시간이면 끝날..."


In [4]:
df = df.melt(
    id_vars=['응답일시', '참여자'],
    value_vars=df.columns[2:],
    var_name='professor',
    value_name='review'
)

# Extract the professor name without the title
df['professor'] = df['professor'].str.replace(' 교수님', '')

df.dropna(inplace=True)

In [5]:
df.head()

,응답일시,참여자,professor,review
0,2025-05-13 18:51:32,1,김유섭,"친근하다,적극적이다,좋다"
2,2025-05-13 19:33:43,3,김유섭,교수님의 말투가 친근하고 또박또박 말씀하셔서 듣기가 좋습니다. 강의 자료만 봐도 이...
3,2025-05-13 20:01:56,4,김유섭,말투가 느리시지만 상냥한 어투로 말해주신다. 강의 자료는 깔끔하고 이해가 쉽게 되어...
6,2025-05-13 20:10:54,7,김유섭,교수님 인상과 말투가 친근하다\n수업이해도 보통이다\n학생 수업 참여 유도 없다
7,2025-05-13 20:47:02,8,김유섭,교수님 설명을 쉽게 잘해주신다. 시험 난이도는 적당한 편이다. 적극적으로 수업 하신...


In [6]:
# 결측값 처리 전
df['professor'].value_counts()

,count
professor,
김유섭,43
양은샘,34
김은주,33
이정근,33
신미영,31
김선정,29


### Generate embeddings

In [7]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# 1) 임베딩 모델 로드 (가볍고 빠른 모델 추천)
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 2) 기준 문장 리스트 (수업 안 들음/모름 의미)
reference_texts = [
    '수업을 들은 적이 없다',
    '수강한 적 없음',
    '기억이 안 난다',
    '모름',
    '머름',
    'ㅡ'
]

# 기준 문장 임베딩
ref_embeddings = model.encode(reference_texts, convert_to_tensor=True)

def to_nan_if_similar(text, threshold=0.55):
    emb = model.encode(text, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(emb, ref_embeddings)
    max_score = cosine_scores.max().item()
    if max_score > threshold:
        print(f'NaN 처리됨: "{text}", 유사도: {max_score}')
        return np.nan
    return text

# professor_cols는 교수님별 컬럼 리스트
df['review'] = df['review'].apply(to_nan_if_similar)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NaN 처리됨: "잘 기억이 안난다.", 유사도: 0.8921089172363281
NaN 처리됨: "모름", 유사도: 0.9999998211860657
NaN 처리됨: "수업을 들은 적이 없다.", 유사도: 0.9760663509368896
NaN 처리됨: "잘 모릅니다", 유사도: 0.5811394453048706
NaN 처리됨: "x(양은샘교수님 수업 들은적 없는것같음)", 유사도: 0.6181110143661499
NaN 처리됨: "오래돼서 기억이 안난다", 유사도: 0.8561017513275146
NaN 처리됨: "모른다", 유사도: 0.6024789810180664
NaN 처리됨: "수강한적없음", 유사도: 0.9111709594726562
NaN 처리됨: "들은 적이 없습니다", 유사도: 0.6363896131515503
NaN 처리됨: "수업 들어본적 없음", 유사도: 0.8043424487113953
NaN 처리됨: "수업을 들은 적이 없다.", 유사도: 0.9760663509368896
NaN 처리됨: "잘 모릅니다", 유사도: 0.5811394453048706
NaN 처리됨: "오래돼서 기억이 안남.", 유사도: 0.7369534969329834
NaN 처리됨: "위 교수님의 수업을 들은 적 없음", 유사도: 0.7422499656677246
NaN 처리됨: "한번도 수업을 들어보지않았다", 유사도: 0.8047888278961182
NaN 처리됨: "모른다", 유사도: 0.6024789810180664
NaN 처리됨: "들은 적이 없습니다", 유사도: 0.6363896131515503
NaN 처리됨: "수업을 들어본적이 없어서 잘 모르겠습니다.", 유사도: 0.790483832359314
NaN 처리됨: "수업을 듣지 않아서 잘 모르겠다.", 유사도: 0.764672577381134
NaN 처리됨: "수업을 들어보지 않아서 모르겠다. 하지만 듣기로 많은 학생들을 수용하시고 정말 열정적으로 가르치신다고 들었다.", 유사도: 0.575431

In [8]:
df.dropna(inplace=True)

In [9]:
# 결측값 처리 후
df['professor'].value_counts()

,count
professor,
김유섭,43
이정근,25
양은샘,24
김은주,24
신미영,23
김선정,16


### Save

In [10]:
df.to_csv('survey_reviews.csv', index=False, encoding='utf-8-sig')